# antlr




## introduction



Why do this. Ultimately, it creates a shortcut for including interpreters for any language in these notebooks. Less reliance on other code, and also converting a library from one framework to another.



In [ ]:
// placeholder for readme.md?


## multilungual abstract syntax trees

This is great day for translating between languages.


### select antlr tree

Using antlr templates because look at all the glorious grammars posted right there on it's main Github page! https://github.com/antlr/grammars-v4



#### the code

select antlr tree?


In [ ]:
var fs = require('fs');
var {CommonTokenStream, InputStream} = require('antlr4');
var importer = require('../Core')
var {
    selectDom,
    evaluateDom,
    treeToHtml,
    htmlToTree
} = importer.import(['select tree', 'tree to html', 'html to tree'])

function getParser(language) {
    // TODO: get parser by language lookup
    const {CLexer} = require('../Resources/Parsers/c/CLexer.js')
    const {CParser} = require('../Resources/Parsers/c/CParser.js')

    return (input) => {
        //const parser = new GenericParser()
        const lexer = new CLexer(new InputStream(input))
        const parser = new CParser(new CommonTokenStream(lexer))
        parser.buildParseTrees = true;
        return parser
    }
}

//TODO: fix this with own transpiler because no one is going to share their hard work on StringTemplate4 code
function toString(ctx, subctx) {
    var ast = (subctx || ctx)
    if(typeof ast.ownerDocument !== 'undefined') {
        ast = htmlToTree(ast)
    }
    //escodegen.attachComments(ast, ast.comments, ast.tokens, ast.whitespace)
    return escodegen.generate(ast, {
        comment: true,
        tokens: true,
        whitespace: true,
        format: {indent: {style: '    '}, quotes: 'single'}
    })
}

function selectCode(descriptor, code, language) {
    if(typeof code === 'function') {
        code = code.toString()
    }
    if(typeof code === 'string') {
        if(fs.existsSync(code)) {
            code = fs.readFileSync(code).toString('utf8')
            if(typeof language === 'undefined') {
                // TODO: use language file extension from media-server code module gheshi support
            //    language = 
            }
        }
        var parser = getParser(language)
        try {
            code = parser(code)
            return code
        } catch(e) {
            console.log(e)
            throw new Error('Could not parse \n' + code)
        }
    }
    // No more selectTree because jsel and friends are inconsistent/partially implemented.
    if(typeof code.evaluate === 'undefined'
       && typeof code.ownerDocument === 'undefined') {
        code = selectDom('//Program', treeToHtml(code))
        code.ownerDocument.toString = toString.bind(code, code)
    }
    return selectDom(descriptor, code)
}

module.exports = {
    selectCode
}


#### TODO: test select jison on some quake 3 C code



In [1]:
var antlr = require('antlr4')
var importer = require('../Core')
var {selectCode} = importer.import('select antlr tree')

var tree = selectCode('*', '/Users/briancullinan/planet_quake/code/q3_ui/ui_cdkey.c')
console.log(tree.compilationUnit())


importing /Users/briancullinan/jupyter_ops/Core/cache.ipynb[1] - 1 cells - cache.ipynb[1]
importing minimatch - 1 cell - minimatch.ipynb[2]
compiling /Users/briancullinan/jupyter_ops/Languages/minimatch.ipynb[2] aka minimatch with module,importer,exports,__dirname,__filename,$$
importing expand - 1 cell - minimatch.ipynb[0]
compiling /Users/briancullinan/jupyter_ops/Languages/minimatch.ipynb[0] aka , expand with module,importer,exports,__dirname,__filename,$$
importing balanced.ipynb[0] - 1 cell - balanced.ipynb[0]
compiling /Users/briancullinan/jupyter_ops/Languages/balanced.ipynb[0] aka balanced with module,importer,exports,__dirname,__filename,$$
importing mini parser - 1 cell - minimatch.ipynb[1]
compiling /Users/briancullinan/jupyter_ops/Languages/minimatch.ipynb[1] aka mini parser with module,importer,exports,__dirname,__filename,$$
importing expand - 1 cell - minimatch.ipynb[0]
importing cache notebook,cell cache,refresh regular cache,update regular cache,update code cell - 5 ce

## antlr tools

Creating lexer and parser from .g4 files. Converting XPath to trees. Convert antlr Tool classes to nodejs so we can load grammars on the fly.



#### TODO: use g4 translator to automatically select entry point, because this is silly




#### TODO: convert antlr tree to json matching esprima format with text locations and tokens lists




In [ ]:

public class Examples {

  private static final Gson PRETTY_PRINT_GSON = new GsonBuilder().setPrettyPrinting().create();
  private static final Gson GSON = new Gson();

  public static String toJson(ParseTree tree) {
    return toJson(tree, true);
  }

  public static String toJson(ParseTree tree, boolean prettyPrint) {
    return prettyPrint ? PRETTY_PRINT_GSON.toJson(toMap(tree)) : GSON.toJson(toMap(tree));
  }

  public static Map<String, Object> toMap(ParseTree tree) {
    Map<String, Object> map = new LinkedHashMap<>();
    traverse(tree, map);
    return map;
  }

  public static void traverse(ParseTree tree, Map<String, Object> map) {

    if (tree instanceof TerminalNodeImpl) {
      Token token = ((TerminalNodeImpl) tree).getSymbol();
      map.put("type", token.getType());
      map.put("text", token.getText());
    }
    else {
      List<Map<String, Object>> children = new ArrayList<>();
      String name = tree.getClass().getSimpleName().replaceAll("Context$", "");
      map.put(Character.toLowerCase(name.charAt(0)) + name.substring(1), children);

      for (int i = 0; i < tree.getChildCount(); i++) {
        Map<String, Object> nested = new LinkedHashMap<>();
        children.add(nested);
        traverse(tree.getChild(i), nested);
      }
    }
  }

  public static void main(String[] args) {
    String source = "(1 + 2) * 3";
    ExpressionLexer lexer = new ExpressionLexer(CharStreams.fromString(source));
    ExpressionParser parser = new ExpressionParser(new CommonTokenStream(lexer));
    System.out.println(toJson(parser.parse()));
  }
}
